In [ ]:
from airflow.models.baseoperator import BaseOperator
from airflow.hooks.sqlite_hook import SqliteHook
from airflow.operators.python import PythonOperator
from airflow import DAG
from datetime import timedelta
from airflow.utils.dates import days_ago 
import pandas as pd

class FileSQLiteTransferHook(SqliteHook):

    def get_pandas_df(self, url_or_path):
      """ Read data to df"""
      return pd.read_csv(url_or_path)

    def insert_df_to_db(self, data):
      """ insert into DB """ 
      data.to_sql('table', con=self.get_conn())

class FileSQLiteTransferOperator(BaseOperator):

    def __init__(self, path, **kwargs):
        super().__init__(**kwargs)
        self.hook = None 
        self.path = path # file path


    def execute(self, context):

        # hook init
        self.hook = FileSQLiteTransferHook()

        # get data
        data = self.hook.get_pandas_df(self.path)

        # create table and insert data
        self.hook.insert_df_to_db(data)

# dag and operator

dag = DAG('dag', schedule_interval=timedelta(days=1), start_date=days_ago(1))

t1 = FileSQLiteTransferOperator(
  task_id='transfer_data', 
  path='https://raw.githubusercontent.com/dm-novikov/stepik_airflow_course/main/data_new/2021-01-04.csv', 
  dag=dag)